In [2]:
import pprint

# Define my own hash function: k mod m
def myhash(key, size):  # key is a text string
    hash_value = 7  # better use a prime number
    for i in range(len(key)):
        hash_value = (hash_value * 31 + ord(key[i])) % size
    return hash_value


class DynamicHashTable:
    def __init__(self, initial_size=8):  # Default table size
        self.size = initial_size
        self.table = [None] * self.size  # Array to hold key-value pairs
        self.num_elements = 0

    def _load_factor(self):
        return self.num_elements / self.size

    def _resize(self, new_size):
        old_table = self.table
        self.size = new_size
        self.table = [None] * self.size
        self.num_elements = 0

        for entry in old_table:
            if entry is not None:
                self.insert(entry[0], entry[1])

    def insert(self, key, value):
        index = myhash(key, self.size)

        while self.table[index] is not None:  # Linear probing
            if self.table[index][0] == key:  # Update existing key
                self.table[index] = (key, value)
                return
            index = (index + 1) % self.size

        self.table[index] = (key, value)
        self.num_elements += 1

        # Check load factor and resize if necessary
        if self._load_factor() >= 4 / 5:
            self._resize(self.size * 2)

    def delete(self, key):
        index = myhash(key, self.size)

        while self.table[index] is not None:
            if self.table[index][0] == key:  # Found key
                self.table[index] = None
                self.num_elements -= 1

                # Rehash the cluster of entries after the removed item
                next_index = (index + 1) % self.size
                while self.table[next_index] is not None:
                    entry = self.table[next_index]
                    self.table[next_index] = None
                    self.num_elements -= 1
                    self.insert(entry[0], entry[1])
                    next_index = (next_index + 1) % self.size

                # Check load factor and resize if necessary
                if self._load_factor() <= 1 / 5 and self.size > 8:  # Minimum size of 8
                    self._resize(self.size // 2)
                return

            index = (index + 1) % self.size

    def get(self, key):
        index = myhash(key, self.size)

        while self.table[index] is not None:
            if self.table[index][0] == key:  # Found key
                return self.table[index][1]
            index = (index + 1) % self.size

        return None  # Key not found

    def __str__(self):
        return pprint.pformat(self.table)


# Example Usage
if __name__ == "__main__":
    capitals = {
        'France': 'Paris',
        'United States': 'Washington D.C.',
        'Italy': 'Rome',
        'Canada': 'Ottawa'
    }

    hashtable = DynamicHashTable()
    for key, value in capitals.items():
        hashtable.insert(key, value)

    print("Initial Table:")
    print(hashtable)
    print(f"The capital of Italy is {hashtable.get('Italy')}.")

    # Testing dynamic resizing
    print("\nAdding more entries to trigger resizing...")
    hashtable.insert('Germany', 'Berlin')
    hashtable.insert('Japan', 'Tokyo')
    hashtable.insert('Australia', 'Canberra')
    hashtable.insert('India', 'New Delhi')
    print(hashtable)

    print("\nDeleting entries to trigger contraction...")
    hashtable.delete('France')
    hashtable.delete('United States')
    print(hashtable)


Initial Table:
[('United States', 'Washington D.C.'),
 None,
 ('France', 'Paris'),
 None,
 ('Italy', 'Rome'),
 ('Canada', 'Ottawa'),
 None,
 None]
The capital of Italy is Rome.

Adding more entries to trigger resizing...
[('India', 'New Delhi'),
 None,
 ('France', 'Paris'),
 None,
 None,
 ('Canada', 'Ottawa'),
 None,
 None,
 ('United States', 'Washington D.C.'),
 ('Australia', 'Canberra'),
 None,
 None,
 ('Italy', 'Rome'),
 ('Germany', 'Berlin'),
 None,
 ('Japan', 'Tokyo')]

Deleting entries to trigger contraction...
[('India', 'New Delhi'),
 None,
 None,
 None,
 None,
 ('Canada', 'Ottawa'),
 None,
 None,
 None,
 ('Australia', 'Canberra'),
 None,
 None,
 ('Italy', 'Rome'),
 ('Germany', 'Berlin'),
 None,
 ('Japan', 'Tokyo')]
